In [1]:
import requests
from io import BytesIO
import zipfile 
import xmltodict
import json
import pymysql
import info

In [2]:
conn = pymysql.connect(host=info.myHost,user=info.myUser,password=info.myPw,db=info.myDb,charset="utf8")
cur = conn.cursor()

In [3]:

api = 'https://opendart.fss.or.kr/api/corpCode.xml'

res = requests.get(api, params={'crtfc_key': info.crtfc_key})
z = zipfile.ZipFile(BytesIO(res.content))
data_xml = z.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

In [4]:

for item in data:
    if item['stock_code'] in ["005930","051910","000660"]: #삼성전자, LG화학, SK하이닉스
        print(item)
        print(item['corp_name'])
        sql = "update company_info set corp_code = '"+item['corp_code']+"',modify_date ='"+item['modify_date']+"' where stock_code='"+item['stock_code']+"'"
        cur.execute(sql)

{'corp_code': '00126380', 'corp_name': '삼성전자', 'stock_code': '005930', 'modify_date': '20201209'}
삼성전자
{'corp_code': '00356361', 'corp_name': 'LG화학', 'stock_code': '051910', 'modify_date': '20210510'}
LG화학
{'corp_code': '00164779', 'corp_name': 'SK하이닉스', 'stock_code': '000660', 'modify_date': '20210511'}
SK하이닉스


In [5]:
conn.commit()
conn.close()